# 池化层

## 实现池化层的正向传播

In [6]:
#实现池化层的正向传播
import torch
from torch import nn
from d2l import torch as d2l

#这里池化和卷积是很类似的，都是通过两个for循环
#x是输入，pool_size是输入窗口大小。model是采用哪种池化方式
def pool2d(X,pool_size,model='max'):
    p_h,p_w=pool_size#这里不用.shape。（2，2）是池化窗口，这是一个元组，没有shape,之前的卷积核是一个矩阵，tensor,所以有矩阵
    Y=torch.zeros((X.shape[0]-p_h+1),(X.shape[1]-p_w+1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if model=='max':
                Y[i,j]=X[i:i+p_h,j:j+p_w].max()
            elif model=='avg':
                Y[i,j]=X[i:i+p_h,j:j+p_w].mean()
    
    return Y

In [7]:
#验证
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
pool2d(X, (2, 2))

tensor([[4., 5.],
        [7., 8.]])

In [8]:
#验证平均池化层
#平均池化层的值通常比最大池化层的值小
pool2d(X, (2, 2), 'avg')

tensor([[2., 3.],
        [5., 6.]])

## 填充和步幅

In [9]:
#先定义一个4*4的矩阵，加入批量大小和通道数
X = torch.arange(16, dtype=torch.float32).reshape((1, 1, 4, 4))
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]]]])

In [10]:
#3*3的窗口
#深度学习框架中步幅与池化窗口大小相同，这样每次看的元素就没有重叠的部分。
pool2d=nn.MaxPool2d(3)
pool2d(X)


tensor([[[[10.]]]])

In [17]:
#手动设定步幅
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X)

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]]]])

In [15]:
#设定一个任意大小的矩形池化窗口，并分别设定填充和步幅的高度和宽度
pool2d = nn.MaxPool2d((2, 3), padding=(1, 1), stride=(2, 3))
pool2d(X)


tensor([[[[ 1.,  3.],
          [ 9., 11.],
          [13., 15.]]]])

In [18]:
#池化层在每个通道上单独作用
#定义了一个两通道的输入
X = torch.cat((X, X + 1), 1)
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]],

         [[ 1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12.],
          [13., 14., 15., 16.]]]])

In [19]:
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]],

         [[ 6.,  8.],
          [14., 16.]]]])

# 答疑

- 池化时窗口重叠与不重叠没有特别大的区别
- python会自动释放没用的
- python计算一次会很慢，如果不确定结果的行数和列数，一般用嵌套list,再转为tensor
- 池化用stride=2特征图尺寸减半，使计算减少，但是也可以放到卷积中。数据会做很多数据增强，使得卷积神经网络看到数据本身发生很多变化，不会过拟合数据的某个位置，卷积层就对位置没有那么敏感，这样就淡化了池化层的使用。
- 正则是为了控制模型的复杂度，